# Pythonで複数のサイトをwebスクレイピング
## webスクレイピング

>webスクレイピング（以下、スクレイピング）とは、webサイトのhtmlからプログラムを使って自動的に情報を抽出、整形することです。定期に特定のサイトを複数閲覧する場合、自動的に情報を取得できるようにすれば非常に効率的です。

## 今回やりたいこと
pythonでスクレイピングした後、取得したデータをメールで送信する。<br>
参考サイト同様、礼拝時間・国際ニュース・某出版社のブログの情報を取得するプログラムを実装しました。

参考サイト: [https://astrophysik928.hatenablog.com/entry/2018/10/14/224152](https://astrophysik928.hatenablog.com/entry/2018/10/14/224152)

## 「BeautifulSoupを使ったスクレイピング」

In [70]:
# import
import urllib.request
from bs4 import BeautifulSoup
import re
import lxml.html
import requests
from time import sleep
import smtplib
from io import StringIO
from email.mime.text import MIMEText
from email import message
import chardet
from config import MAIL
# mail address
from mail import TO_MAIL

### 1. config.py
同階層に```config.py```というファイルを作成してください

```python:config.py
MAIL = {
  "host"      : 'ホスト名',
  "port"      : 'ポート名',
  "from_email": '送信元メールアドレス',
  "username"  : 'ユーザー名（送信元メールアドレスと一緒でいい）',
  "password"  : 'パスワード'
}
```


In [71]:
# config.py
from config import MAIL

### 2. mail.py
同階層に```mail.py```というファイルを作成してください

```python:mail.py
TOMAIL = '宛先メールアドレス'
```

In [72]:
# mail.py
from mail import TO_MAIL

## 1. islamicFinder

In [64]:
# get pray time
def do_pray_time():
    islamic_finder = urllib.request.urlopen("https://www.islamicfinder.org/")
    #ここでwebサイトのhtmlを読み込む
    islamic_soup = BeautifulSoup(islamic_finder, 'html.parser')
    pray_name = [
        u"FAJR", u"SUNRISE", u"DHUHR",
        u"ASR", u"MAGHRIB", u"ISHA", u"QIYAM"
    ]
    #読み込んだhtmlのh1属性の文字列（サイトの見出しに該当）を取得
    pray_header = "//////Today Pray Time//////" + islamic_soup.h1.string + "\n"
    pray_time = pray_header
    #span要素のclass属性が"todayPrayerTime"の全てを取得
    for i, span_element in enumerate(islamic_soup.findAll("span", class_="todayPrayerTime")):
        if(span_element != None):
            #取得したspan要素の文字列（礼拝時間）を取得  
            pray_time = pray_time + \
                pray_name[i] + "　" + span_element.text + "\n"
    pray_time = pray_time + "\n\n\n"
    return pray_time

## 2. iranNews

In [65]:
# get Iran international news site
def do_iran_news():

    res = requests.get("http://parstoday.com/ja")

    #ニュースサイトのhtmlを取得
    iran_international_soup = BeautifulSoup(res.text, 'html.parser')

    iran_news_header = '*** Iran international new site ***' + \
        "\n" + "News ParsToday" + "\n\n"

    iran_news = iran_news_header

    #li要素のclass属性で値が"item item-separator inline-30"の箇所を参照、その全てを取得
    for i, li_element in enumerate(iran_international_soup.findAll("li", class_="item item-separator inline-30")):

        #"item item-separator inline-30"の中のdiv要素及びa属性を参照
        div = li_element.find("div")
        a = li_element.find("a")
        if(div != None):

            #「div.text」で、ニュースランキングのタイトルを、「a.get("href")」で各ニュースへのリンクを取得
            iran_news = iran_news + \
                str(i+1) + "位" + " " + div.text + "\n" + a.get("href") + "\n\n"

    iran_news = iran_news + "\n\n\n"

    return iran_news

In [66]:
# get rokusaisha blog
def do_rokusaisha():
    res2 = urllib.request.urlopen("http://www.rokusaisha.com/index.php")
    
    # ブログのhtmlを取得
    rokusaisha_soup = BeautifulSoup(res2, 'html.parser')
    
    rokusaisha_header = "********" + rokusaisha_soup.title.string + \
        "********" + "\n" + "最新ブログ情報" + "\n\n"
    
    rokusaisha = rokusaisha_header
    
    # id要素のclass属性で値が"ブログタイトル"の箇所を参照、その全てを取得
    for i, td_element in enumerate(rokusaisha_soup.findAll("td", class_="ブログタイトル")):
        if(td_element != None):
            # td要素のテキスト（新着ブログのタイトル）を取得
            rokusaisha = rokusaisha + td_element.text + \
                "\n" + "http://www.rokusaisha.com/blog.php" + "\n\n"
    return rokusaisha

In [67]:
# send mail
def do_mail(information1, information2, information3):
    username = '1484kazuya12@gmail.com'
    password = 'bridgestone12'
    
    # メールの内容を作成
    all_information = information1 + information2 + information3
    
    msg = message.EmailMessage()
    msg.set_content(all_information) # メールの本文
    msg['Subject'] = 'Today information''' # 件名
    msg['From'] = MAIL['from_email'] # メールの送信元
    msg['To'] = TO_MAIL # メール送信先
    
    print("Running")
    
    server = smtplib.SMTP(MAIL['host'], MAIL['port'])
    
    server.ehlo()
    server.starttls()
    server.ehlo()
    server.login(MAIL['username'], MAIL['password'])
    server.send_message(msg)
    server.quit()
    
    print('Sending mail is completed')

In [68]:
if __name__ == '__main__':
    do_mail(do_pray_time(), do_iran_news(), do_rokusaisha())

Running
Sending mail is completed
